In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
from scr.analysis.perlayer import LayerLoss

In [4]:
layer_analysis_dict = LayerLoss(
    input_path="results/train_val_test",
    output_path="results/analysis_layer",
).layer_analysis_dict

In [6]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-fixa",
    output_path="results/sklearn-scaley-fixa_layer",
).layer_analysis_dict

In [32]:
layer_analysis_dict.keys(), layer_analysis_dict["proeng_gb1_low_vs_high_mean"].keys(), layer_analysis_dict["proeng_gb1_low_vs_high_mean"]["esm1b_t33_650M_UR50S"].keys()

(dict_keys(['proeng_gb1_two_vs_rest_mean', 'proeng_gb1_low_vs_high_mean', 'proeng_aav_one_vs_many_mean']),
 dict_keys(['esm1b_t33_650M_UR50S', 'esm1_t6_43M_UR50S', 'esm1_t12_85M_UR50S', 'esm1_t34_670M_UR50S']),
 dict_keys(['train_mse', 'test_ndcg', 'test_rho']))